In [1]:
import tensorflow as tf
from tensorflow.keras import Model

In [2]:
class SimpleModel(Model):
    
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.relu = tf.keras.layers.ReLU()
        self.fc1 = tf.keras.layers.Dense(128, use_bias=True)
        self.fc2 = tf.keras.layers.Dense(64, use_bias=True)
        self.fc3 = tf.keras.layers.Dense(10)
    
    def call(self, in_tensor):
        out = self.flatten(in_tensor)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return self.fc3(out)

In [3]:
model = SimpleModel()

In [4]:
import tensorflow_datasets as tfds

In [5]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    download=True,
    with_info=True
)

In [6]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., tf.one_hot(label, 10)

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
model.build((None, 28, 28, 1))

In [9]:
model.summary()

Model: "simple_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  8256      
_________________________________________________________________
dense_2 (Dense)              multiple                  650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=['accuracy'],
)

In [11]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 2s 4ms/step - loss: 0.2358 - accuracy: 0.9278 - val_loss: 0.1359 - val_accuracy: 0.9583
Epoch 2/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1192 - accuracy: 0.9638 - val_loss: 0.1218 - val_accuracy: 0.9660
Epoch 3/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0986 - accuracy: 0.9708 - val_loss: 0.1646 - val_accuracy: 0.9513
Epoch 4/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0849 - accuracy: 0.9748 - val_loss: 0.1307 - val_accuracy: 0.9621
Epoch 5/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0778 - accuracy: 0.9779 - val_loss: 0.1301 - val_accuracy: 0.9664
Epoch 6/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0683 - accuracy: 0.9795 - val_loss: 0.1299 - val_accuracy: 0.9702


In [13]:
model.save('keras_model/')

INFO:tensorflow:Assets written to: keras_model/assets


INFO:tensorflow:Assets written to: keras_model/assets


In [14]:
def represent_ds():
    for _ in range(1024):
        yield [
            tf.random.uniform((1, 28, 28, 1), 0, 1, dtype=tf.float32)
        ]

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_ds

In [16]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [18]:
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec((None, 28, 28, 1), dtype=tf.float32))
model_func = convert_variables_to_constants_v2(model_func, lower_control_flow=False)

In [19]:
with open('keras_model/const_graph.pb', 'wb') as fid:
    fid.write(model_func.graph.as_graph_def().SerializeToString())

In [20]:
tflite_content = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/5l/jrch3z5x6jx6tcz33qkv1mgh0000gn/T/tmp0g_3t0m0/assets


INFO:tensorflow:Assets written to: /var/folders/5l/jrch3z5x6jx6tcz33qkv1mgh0000gn/T/tmp0g_3t0m0/assets


In [21]:
with open('./keras_model.tflite', 'wb') as fid:
    fid.write(tflite_content)